# ⚛️ VQE Ground State Simulation of H₂ with Optimizer Comparison (Noiseless)

This notebook implements the **Variational Quantum Eigensolver (VQE)** to compute the **ground state energy of dihydrogen (H₂)** using a **noiseless quantum simulator**. It compares multiple classical optimizers to evaluate their performance in minimizing the energy landscape of a quantum chemistry problem.

### Key Features:
- **Molecular Setup**: Constructs the molecular Hamiltonian for H₂ using the STO-3G basis set.
- **Excitation Ansatz**: Uses a UCC-style ansatz incorporating both **single and double excitations**.
- **Optimizer Evaluation**:
  - Runs the VQE algorithm using a range of optimizers defined in `vqe_utils.OPTIMIZERS`.
  - Tracks and compares convergence behavior across optimizers.
- **Ground State Analysis**:
  - Identifies the optimizer yielding the **lowest final energy**.
  - Decomposes the resulting quantum state into basis states with corresponding amplitudes.
  - Visualizes the ground state amplitudes in a bar plot.

### 📊 Visualization Outputs:
- Energy convergence plots for each optimizer.
- Final ground state quantum state expressed in ket notation.
- Bar chart of basis state amplitudes.

### Purpose:
To benchmark classical optimizers within the VQE framework on a minimal chemistry system (H₂), helping guide the selection of optimizers for more complex molecular systems.

In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import json, os
from vqe_utils import get_optimizer, excitation_ansatz, OPTIMIZERS, set_seed, make_run_config_dict, run_signature, find_existing_run, build_run_filename

set_seed(0)                              # Reproducible runs
seed = 0
IMG_DIR = "images"                       # Single image directory used repo-wide
os.makedirs(IMG_DIR, exist_ok=True)
RUNS_DIR = "runs"                        # Run record directory used repo-wide
os.makedirs(RUNS_DIR, exist_ok=True)

In [ ]:
# Define the atoms in the dihydrogen molecule
symbols = ["H", "H"]

# Define the coordinates (in Ångströms)
coordinates = np.array([
    [0.0000, 0.0000, 0.0000],  # Hydrogen 1
    [0.0000, 0.0000, 0.7414]   # Hydrogen 2
])

# Define the dihydrogen Hamiltonian and the number of qubits required
# Default STO-3G basis set
basis = "STO-3G"
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0, basis=basis)

# 1 spatial orbital for each H
# Each spatial orbital is doubly-degenerate (2 spin orbitals)
print(f"{qubits} qubits required")

In [ ]:
# Which orbitals are occupied (1) or unoccupied (0) in the mean-field (Hartree-Fock) solution
electrons = 2  # 1 from each H
hf = qchem.hf_state(electrons=electrons, orbitals=qubits)

# Show that the first 2 orbitals are occupied
# Print the state in bra-ket notation (leftmost bit = highest-index spin orbital)
print(f"Hartree-Fock state: {'|' + ''.join(str(bit) for bit in hf) + '⟩'}")

In [ ]:
# Define the number of required quantum wires / logical qubits
num_wires = qubits

# Create quantum device simulator backend
dev = qml.device("default.qubit",  # Noiseless state vector simulator
                 wires=num_wires)


@qml.qnode(dev)  # Transforms exp_energy below into a quantum node
def exp_energy(state):
    qml.BasisState(np.array(state, dtype=int), wires=range(num_wires))

    # Return the expectation value of the molecular Hamiltonian
    return qml.expval(hamiltonian)

# Calculate ⟨ψ_hf| hamiltonian |ψ_hf⟩ in Hartree (Ha) units
# 1 Ha = 27.2 eV
exp_energy(hf)

In [ ]:
# Generate excitation indices
singles, doubles = excitations(electrons, qubits)
print(f"Singles Excitations: {singles}")  # e.g: [0, 1] is a single excitation of an electron in orbital 0 to orbital 1
print(f"Doubles Excitations: {doubles}")  # e.g: [0, 1, 2, 3] is a double excitation of electrons in orbitals 0 and 1 to orbitals 2 and 3

In [ ]:
ansatz_desc = "UCC singles+doubles (excitation_ansatz with (singles, doubles))"

# Define the VQE cost function
@qml.qnode(dev)
def cost_function(params):
    excitation_ansatz(params, wires=range(num_wires), hf_state=hf,
                      excitations=(singles, doubles), excitation_type="both")
    
    # Measure the expectation value of the Hamiltonian after applying the ansatz:
    # E(theta) = ⟨ψ(theta)| H |ψ(theta)⟩
    return qml.expval(hamiltonian)

def run_vqe_with_cache(opt_name, stepsize, max_iterations):
    # Build configuration and signature (no circuit calls here)
    cfg = make_run_config_dict(
        symbols=symbols,
        coordinates=coordinates,
        basis=basis,
        ansatz_desc=ansatz_desc,
        optimizer_name=opt_name,
        stepsize=stepsize,
        max_iterations=max_iterations,
        seed=seed,
    )

    # Canonical file name: readable prefix and signature
    sig = run_signature(cfg)
    fname = os.path.join(RUNS_DIR, f"H2_Noiseless_{opt_name}_s{seed}__{sig}.json")
    existing = find_existing_run(RUNS_DIR, sig)

    if existing:
        # Load results if this configuration has been run before
        with open(existing) as f:
            rec = json.load(f)
        print(f"[reuse] {opt_name}: loaded {existing}")
        energy = rec["energies"]
        if "final_params" in rec:
            params = np.array(rec["final_params"], requires_grad=True)
        elif "params_by_step" in rec and rec["params_by_step"]:
            params = np.array(rec["params_by_step"][-1], requires_grad=True)
        else:
            print(f"[reuse] {opt_name}: no params stored; recomputing.")
            existing = None
    else:
        print(f"[compute] {opt_name}: no matching run found; optimizing now.")

    if not existing:
        # Compute results if this configuration has NOT been ran before
        opt = get_optimizer(opt_name, stepsize=stepsize)
        params = np.zeros(len(singles) + len(doubles), requires_grad=True)
        energy = [cost_function(params)]
        for _ in range(max_iterations):
            params, e_prev = opt.step_and_cost(cost_function, params)
            energy.append(e_prev)
        energy[-1] = float(cost_function(params))

        run_record = {
            "timestamp": __import__("time").strftime("%Y-%m-%d %H:%M:%S"),
            "molecule": "H2",
            "symbols": symbols,
            "geometry": coordinates.tolist(),
            "basis": basis,
            "electrons": electrons,
            "num_wires": num_wires,
            "ansatz": ansatz_desc,
            "optimizer": {
                "name": opt_name,
                "stepsize": stepsize,
                "iterations_planned": max_iterations,
                "iterations_ran": len(energy) - 1,
            },
            "seed": seed,
            "energies": [float(e) for e in energy],
            "final_params": [float(x) for x in params],
            "config_hash": sig,
        }

        with open(fname, "w") as f:
            json.dump(run_record, f, indent=2)
        print(f"[saved] {opt_name}: {fname}")

    return {"energy": energy, "params": params}

# Run (or load) VQE for all optimizers
results = {}
max_iterations = 50
stepsize = 0.1

for name in OPTIMIZERS:
    print(f"Running VQE with {name}...")
    results[name] = run_vqe_with_cache(name, stepsize=stepsize, max_iterations=max_iterations)

In [ ]:
# Plot comparison of all optimizer energy curves
plt.figure(figsize=(10, 6))
for name, res in results.items():
    energies = res["energy"]
    plt.plot(range(len(energies)), energies, label=name)

plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("H₂ VQE: Optimizer Comparison (Noiseless)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/H2_Optimizer_Comparison.png", dpi=300)
plt.show()

In [ ]:
for name, res in results.items():
    final_energy = res["energy"][-1]
    print(f"{name}:\nFinal ground state energy = {final_energy:.8f} Ha\n")

In [ ]:
# Choose optimizer with lowest final energy
# If ties occur at machine precision, this picks the first min encountered.
best_name = min(results, key=lambda n: results[n]["energy"][-1])
best_params = results[best_name]["params"]
print(f"Using best optimizer: {best_name}")


@qml.qnode(dev)
def ground_state(params):
    excitation_ansatz(params, wires=range(num_wires), hf_state=hf,
                      excitations=(singles, doubles), excitation_type="both")
    
    # Return the entire quantum statevector
    return qml.state()

final_state = ground_state(best_params)

# Find non-zero (or near non-zero) entries
threshold = 1e-2  # Recommended smaller threshold to capture all significant amplitudes
non_zero_indices = np.where(np.abs(final_state) > threshold)[0]
non_zero_amplitudes = final_state[non_zero_indices]

# Build the full ket notation string
ket_terms = []
for idx, amp in zip(non_zero_indices, non_zero_amplitudes):
    binary_state = f"|{idx:0{num_wires}b}⟩"
    amp_str = f"{amp.real:.4f}" if np.isclose(amp.imag, 0, atol=1e-6) else f"({amp.real:.4f} + {amp.imag:.4f}j)"
    ket_terms.append(f"{amp_str}{binary_state}")

ket_notation = " + ".join(ket_terms)
print(f"Ground state of H₂:\n|ψ⟩ = {ket_notation}")

In [ ]:
# Prepare labels and amplitudes for the plot
labels = [f"|{i:0{num_wires}b}⟩" for i in non_zero_indices]
amplitudes = np.abs(non_zero_amplitudes)

# Bar plot
plt.figure(figsize=(12, 6))
plt.bar(labels, amplitudes)
plt.xlabel('Basis States', fontsize=14)
plt.ylabel('Amplitude', fontsize=14)
plt.title('Ground State of H₂ (VQE Result)', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/H2_Ground_State.png", dpi=300)
plt.show()